In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [ ]:
#Classificador LDA
class LDA:
    def __init__(self):
        self.mean = None
        self.cov = None
        self.prior = None
        self.classes = None

    def fit(self, X, y):
        self.classes = np.unique(y)
        self.mean = np.zeros((len(self.classes), X.shape[1]))
        self.cov = np.zeros((X.shape[1], X.shape[1]))
        self.prior = np.zeros(len(self.classes))

        for i, c in enumerate(self.classes):
            Xc = X[y == c]
            self.mean[i] = np.mean(Xc, axis=0)
            self.cov += np.cov(Xc.T) * (len(Xc) - 1)
            self.prior[i] = len(Xc) / len(X)

        self.cov /= len(X) - len(self.classes)

    def predict(self, X):
        y_pred = np.zeros(len(X))

        for i, x in enumerate(X):
            posteriors = []
            for j, c in enumerate(self.classes):
                prior = self.prior[j]
                mean = self.mean[j]
                cov = self.cov
                likelihood = np.exp(-0.5 * (x - mean).T @ np.linalg.inv(cov) @ (x - mean))
                posterior = prior * likelihood
                posteriors.append(posterior)

            y_pred[i] = self.classes[np.argmax(posteriors)]

        return y_pred

In [ ]:
#Classificador QDA
class QDA:
    def __init__(self):
        self.mean = None
        self.cov = None
        self.prior = None
        self.classes = None

    def fit(self, X, y):
        self.classes = np.unique(y)
        self.mean = np.zeros((len(self.classes), X.shape[1]))
        self.cov = np.zeros((len(self.classes), X.shape[1], X.shape[1]))
        self.prior = np.zeros(len(self.classes))

        for i, c in enumerate(self.classes):
            Xc = X[y == c]
            self.mean[i] = np.mean(Xc, axis=0)
            self.cov[i] = np.cov(Xc.T) * (len(Xc) - 1)
            self.prior[i] = len(Xc) / len(X)

    def predict(self, X):
        y_pred = np.zeros(len(X))

        for i, x in enumerate(X):
            posteriors = []
            for j, c in enumerate(self.classes):
                prior = self.prior[j]
                mean = self.mean[j]
                cov = self.cov[j]
                likelihood = np.exp(-0.5 * (x - mean).T @ np.linalg.inv(cov) @ (x - mean))
                posterior = prior * likelihood
                posteriors.append(posterior)

            y_pred[i] = self.classes[np.argmax(posteriors)]

        return y_pred

In [ ]:
#Utilizando a base de dados iris
iris = datasets.load_iris()
X = iris.data
y = iris.target

#Separando em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#Normalização dos dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

lda = LDA()
lda.fit(X_train, y_train)
y_pred_train = lda.predict(X_train)
y_pred_test = lda.predict(X_test)

#Acurária do treino e teste com LDA
print('------LDA------')
print('Acurácia no conjunto de treino:', accuracy_score(y_train, y_pred_train))
print('Acurácia no conjunto de teste:', accuracy_score(y_test, y_pred_test))

qda = QDA()
qda.fit(X_train, y_train)
y_pred_train = qda.predict(X_train)
y_pred_test = qda.predict(X_test)

#Acurácia do treino e teste com QDA
print('------QDA------')
print('Acurácia no conjunto de treino:', accuracy_score(y_train, y_pred_train))
print('Acurácia no conjunto de teste:', accuracy_score(y_test, y_pred_test))

------LDA------
Acurácia no conjunto de treino: 0.9714285714285714
Acurácia no conjunto de teste: 1.0
------QDA------
Acurácia no conjunto de treino: 0.9714285714285714
Acurácia no conjunto de teste: 0.9777777777777777
